In [1]:
import cv2

def convert_color(img, mode="gray"):
    if mode == "gray":
        return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    elif mode == "hsv":
        return cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    elif mode == "rgb":
        return cv2.cvtColor(img, cv2.COLOR_HSV2BGR)

## Contrast and Brightness

In [2]:
def adjust_contrast_brightness(img, alpha=1.0, beta=0):
    # alpha: contrast [1.0-3.0], beta: brightness [0-100]
    return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)


## Histogram

In [3]:
import matplotlib.pyplot as plt

def show_histogram(img):
    color = ('b','g','r')
    for i,col in enumerate(color):
        hist = cv2.calcHist([img],[i],None,[256],[0,256])
        plt.plot(hist,color=col)
    plt.show()


## Gaussian & Bilateral Filters

In [4]:
def gaussian_filter(img, ksize=5, sigma=1.0):
    return cv2.GaussianBlur(img, (ksize, ksize), sigma)

def bilateral_filter(img, d=9, sigmaColor=75, sigmaSpace=75):
    return cv2.bilateralFilter(img, d, sigmaColor, sigmaSpace)


## Edge & Line Detection

In [5]:
def canny_edge(img, low=50, high=150):
    return cv2.Canny(img, low, high)

def hough_lines(img):
    edges = canny_edge(img)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)
    result = img.copy()
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line[0]
            cv2.line(result, (x1,y1), (x2,y2), (0,255,0), 2)
    return result


## Panorama

In [7]:
def create_panorama(img1, img2):
    orb = cv2.ORB_create()
    k1, d1 = orb.detectAndCompute(img1, None)
    k2, d2 = orb.detectAndCompute(img2, None)

    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(d1, d2)
    matches = sorted(matches, key=lambda x:x.distance)
    pts1 = np.float32([k1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
    pts2 = np.float32([k2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
    H, _ = cv2.findHomography(pts2, pts1, cv2.RANSAC)
    result = cv2.warpPerspective(img2, H, (img1.shape[1]+img2.shape[1], img1.shape[0]))
    result[0:img1.shape[0], 0:img1.shape[1]] = img1
    return result


## Translation, Rotation, Scale

In [8]:
def transform_image(img, tx=50, ty=50, angle=45, scale=1.0):
    rows, cols = img.shape[:2]
    M_translate = np.float32([[1, 0, tx], [0, 1, ty]])
    M_rotate = cv2.getRotationMatrix2D((cols/2, rows/2), angle, scale)
    translated = cv2.warpAffine(img, M_translate, (cols, rows))
    rotated = cv2.warpAffine(translated, M_rotate, (cols, rows))
    return rotated


## Calibrate the Camera

In [9]:
def calibrate_camera(chessboard_images, pattern_size=(9,6)):
    objp = np.zeros((pattern_size[0]*pattern_size[1], 3), np.float32)
    objp[:,:2] = np.mgrid[0:pattern_size[0],0:pattern_size[1]].T.reshape(-1,2)
    objpoints, imgpoints = [], []

    for img in chessboard_images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    return mtx, dist


In [9]:
# replace pywavefront loading with:
import numpy as np
vertices = np.array([
    [0, 0, 0],
    [0, 1, 0],
    [1, 1, 0],
    [1, 0, 0],
    [0, 0, 1],
    [0, 1, 1],
    [1, 1, 1],
    [1, 0, 1],
], dtype=np.float32)


In [10]:
import os

print(os.getcwd())
print(os.path.exists("model.obj"))

c:\Users\User\OneDrive\Desktop\cv lab 2025\Assignment_1
True


## Saved AR_Overlay.png

In [11]:
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
marker_id = 0


In [12]:
import cv2
import numpy as np

# --- Load calibration data ---
data = np.load('calibration.npz')
mtx, dist = data['mtx'], data['dist']

# --- Initialize camera ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Cannot open camera")
    exit()

# --- ArUco marker info ---
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
detector_params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, detector_params)

marker_length = 0.05  # Marker size in meters (5 cm if printed at that size)

print("🎥 Show your A4 ArUco marker (ID=0) to the camera. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # --- Detect markers ---
    corners, ids, rejected = detector.detectMarkers(gray)

    if ids is not None and len(ids) > 0:
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        # --- Estimate pose ---
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(
            corners, marker_length, mtx, dist)

        for rvec, tvec in zip(rvecs, tvecs):
            cv2.aruco.drawAxis(frame, mtx, dist, rvec, tvec, 0.03)

        cv2.putText(frame, "✅ Marker Detected", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        cv2.putText(frame, "🔍 Searching for Marker...", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("AR Overlay", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎥 Show your A4 ArUco marker (ID=0) to the camera. Press 'q' to quit.


In [14]:
# Load image
frame = cv2.imread('A4_ArUco_Marker.png')

# Convert to grayscale
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Load predefined dictionary and parameters
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

# Detect the marker corners
corners, ids, rejected = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

if ids is not None:
    # Draw marker boundaries
    cv2.aruco.drawDetectedMarkers(frame, corners, ids)

    # Estimate pose of the marker (assuming marker side length = 0.05m)
    marker_length = 0.05
    rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, marker_length, mtx, dist)

    # Draw the axis for visualization
    for rvec, tvec in zip(rvecs, tvecs):
        cv2.aruco.drawAxis(frame, mtx, dist, rvec, tvec, 0.03)

cv2.imshow("Marker Detection", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


AttributeError: module 'cv2.aruco' has no attribute 'Dictionary_get'

Load trex_model

In [6]:
import cv2
import numpy as np
import pywavefront  # to load .obj files

# Load the .obj model
scene = pywavefront.Wavefront('trex_model.obj', collect_faces=True)

# Function to project vertices onto the image
def project_vertices(vertices, rvec, tvec, camera_matrix, dist_coeffs):
    projected_points, _ = cv2.projectPoints(vertices, rvec, tvec, camera_matrix, dist_coeffs)
    return projected_points.reshape(-1, 2)

# Draw the model
for mesh in scene.mesh_list:
    for face in mesh.faces:
        pts = np.array([scene.vertices[i] for i in face], dtype=np.float32)
        pts = pts * 0.5  # scale up or down to fit AR scene
        img_pts = project_vertices(pts, rvec, tvec, camera_matrix, dist_coeffs)
        img_pts = img_pts.astype(int)
        cv2.polylines(frame, [img_pts], isClosed=True, color=(0, 0, 255), thickness=2)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\OneDrive\\Desktop\\cv lab 2025\\Assignment_1\\1.mtl'